# Laboratoire facultatif 
## Ce carnet web IPython ne fonctionne pas dans Google Colab mais il peut s'exécuter sur un poste de travail sur lequel est installé un fureteur piloté par Selenium.

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Moissonnage de données sur la Toile - formulaire web

## Les marées des sept prochains jours à Tadoussac 

<p>Vous planifiez une sortie en kayak de mer à Tadoussac qui est située sur le fleuve Saint-Laurent à l'embouchure du Saguenay. Vous allez moissonner les informations concernant les marées pour les sept prochains jours à compter du 21 septembre 2021. Pour cela, vous utiliserez un formulaire disponible sur le site de Pêches et Océans Canada. </p> 

<p>Pour simuler les interactions que fait un humain avec le site web, vous utiliserez la bibliothèque python <a href="https://selenium-python.readthedocs.io/" target='_blank'>Selenium</a>. L'extraction des sections pertinentes de la page web des résultats sera faite avec l'outil python <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a>. Enfin, des <a href="https://docs.python.org/fr/3/howto/regex.html" target='_blank'>expressions régulières<sup>1</sup></a> seront utilisées pour extraire les informations détaillées.</p>
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - pratique: </b><a href="https://regex101.com/" target='_blank'>regex101.com</a> est un excellent site pour pratiquer et mettre au point des expressions régulières.</span>

## Importation des bibliothèques Python
<ul>
<ul>
  <li><b><i>re</i></b> (expressions régulières), <b><i>json</i></b> (traitement du format de données json: JavaScript Object Notation) et et <b><i>datetime</i></b> (traitement des dates et de l'heure) sont des bibliothèques Python standard, vous n'avez pas besoin de les installer, il suffit de les importer.</li>
    <li>Installation d'un <a href="https://selenium-python.readthedocs.io/installation.html#drivers" target='_blank'>pilote pour le contrôle de votre fureteur par Selenium</a></li>
  <ul>
      <li><a href="https://sites.google.com/a/chromium.org/chromedriver/downloads" target='_blank'>Pilotes</a> pour le fureteur Chrome</li> 
      <li>Par exemple sur Mac OS et le fureteur Chrome, brew install chromedriver</li>
  </ul>
  <li>Installation de <a href="https://selenium-python.readthedocs.io/" target='_blank'>Selenium</a></li>
  <ul>
    <li>Par exemple, sudo pip3 install selenium</li>
  </ul>
  <li>Installation de <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a></li>
  <ul>
    <li>Par exemple, sudo pip3 install bs4</li>
  </ul>
</ul>
</ul>

In [1]:
# -*- coding: utf-8 -*-
# Installer la bibliothèque BeautifulSoup de manipulation du HTML  
# sudo pip3 install bs4
from bs4 import BeautifulSoup

# Installer le pilote ChromeDriver ou autre pour piloter le fureteur avec Selenium
# en fonction du fureteur utilisé
# https://selenium-python.readthedocs.io/installation.html#drivers

# Installer Selenium
# sudo pip3 install selenium

# Attention! L'installation de webdriver-manager ne semble pas fonctionner pour tous les système d'exploitation
# sudo pip3 install webdriver-manager 

from selenium import webdriver

import re
import pandas as pd

print("Bibliothèques Python importées")

Bibliothèques Python importées


## Analyse des points d'entrée du formulaire avec la fonction d'inspection de votre fureteur
<ul>
    <ul>
        <li>Ouvrir la page du <a href="https://www.marees.gc.ca/fra/station?sid=2985" target='_blank'>formulaire</a> sur le site de Pêches et Océans Canada;</li>
        <li>Reconstituer la séquence des interactions avec le formulaire (clics, menus et saisie clavier);</li>
        <li>Obtenir les identifiants des boutons, des menus, des champs de saisie;</li>
    </ul>
</ul>   

## Programmation d'un script d'interaction Selenium avec le formulaire des marées



In [106]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from warnings import filterwarnings
filterwarnings("ignore")

# Instanciation d'un pilote pour le fureteur Chrome - méthode obsolète
# Il faut d'abord télécharger le pilote sur https://sites.google.com/chromium.org/driver/
pilote_chrome = webdriver.Chrome("/usr/local/bin/chromedriver")

# La nouvelle façon d'installer le pilote pour le fureteur Chrome
# mais pour cela, il faut réussir à installer la bibliothèque webdriver_manager
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
#
# pilote_chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


# Ouvrir la page web où se trouve le formulaire des marées
pilote_chrome.get("https://www.marees.gc.ca/fr/stations/")

localite = "Tadoussac"

# Demander le formulaire pour la localité de Tadoussac avec l'outil de recherche / filtrage
saisie_localite = pilote_chrome.find_element(By.XPATH, "//div/label/input")
saisie_localite.clear()
saisie_localite.send_keys(localite)

# Demander un délai d'exécution
import time 
time.sleep(2)

# Cliquer l'hyperlien vers la page des marés pour Tadoussac
hyperlien_localite = pilote_chrome.find_element(By.XPATH, "//td/a")
hyperlien_localite.click()

# Demander un délai d'exécution
time.sleep(2)

# Changer la date du composant calendrier, ici le 21 septembre 2023
saisie_date = pilote_chrome.find_element(By.ID, "water-levels-date")
saisie_date.clear()
pilote_chrome.execute_script("arguments[0].value = '2023-09-21';", saisie_date)
# Soumettre la requête pour obtenir l'horaire des marées à Tadoussac pour
# les sept procahins jours à compter du 21 septembre 2023
bouton_soumettre = pilote_chrome.find_element(By.ID, "water-levels-date-submit")
bouton_soumettre.click()

# Demander un délai d'exécution
time.sleep(2)

# Obtenir la page web des résultats
page_reponse = pilote_chrome.page_source

print("Le script Selenium devrait ouvrir un fureteur et afficher un message")
print("du genre: « Browser is being controlled by automated test software. »")
print("puis le script Selenium simule les interactions avec le formulaire.\n")
print("Script Selenium exécuté, formulaire soumis, page de résultats récupérée...")

Le script Selenium devrait ouvrir un fureteur et afficher un message
du genre: « Browser is being controlled by automated test software. »
puis le script Selenium simule les interactions avec le formulaire.

Script Selenium exécuté, formulaire soumis, page de résultats récupérée...


## Analyse de la page de résultats retournée avec `BeautifulSoup`

### Affichage formaté de la page de résultats en HTML 

In [113]:
# Analyser la page de réponse retournée avec BeautifulSoup
dom_page_resultats = BeautifulSoup(page_reponse,"html.parser") 

# Afficher la page web retournée
print(dom_page_resultats.prettify())

<html class="js js backgroundsize borderimage csstransitions fontface svg details progressbar meter no-mathml cors largeview wb-enable" dir="ltr" lang="fr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/i18n/fr.min.js">
  </script>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/deps/jquery.dataTables.min.js">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js">
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i["GoogleAnalyticsObject"]=r;i[r]=i[r]||function(){(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*n

## Extraction des sections pertinentes de la page de résultats avec `Beautiful Soup`

### Affichage des sections pertinentes contenant les données de marée pour sept jours

In [118]:
# Extraire de la page les sections pertinentes
# cela implique un examen du contenu de la page et
# une connaissance des possibilités de BeautifulSoup
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

# Extraire les paramètres de la requête de recherche d'information
parametres_bruts = dom_page_resultats.find_all("table", {"class":"day-tables clearfix table-bordered responsive-enabled table table-hover table-striped"})
print("Sections pertinentes en HTML:\n")
print("Nombre de jours:",len(parametres_bruts),"\n")
print(parametres_bruts)
# parametres_bruts = parametres_bruts.find("div", {"class":"wb-eqht wb-init wb-eqht-inited"})


Paramètres bruts:

Nombre de jours: 7 

[<table aria-label="Times and Heights for High and Low Tides for 2023-09-21" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2023-09-21" style="font-size: 0.8em">
<caption>2023-09-21 (Jeu)</caption>
<thead>
<tr>
<th>Heure EDT</th>
<th>Niveaux (m)</th>
<th>Niveaux (pi)</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>01:02</td>
<td>0.9</td>
<td>2.9</td>
</tr>
<tr class="even">
<td>07:08</td>
<td>3.5</td>
<td>11.4</td>
</tr>
<tr class="odd">
<td>12:54</td>
<td>1.1</td>
<td>3.5</td>
</tr>
<tr class="even">
<td>19:27</td>
<td>4.1</td>
<td>13.4</td>
</tr>
</tbody>
</table>, <table aria-label="Times and Heights for High and Low Tides for 2023-09-22" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2023-09-22" style="font-size: 0.8em">
<caption>2023-09-22 (ven)</caption>
<thead>
<tr>
<th>Heure EDT</th

### Affichage du texte brut sans les balises HTML

In [122]:
print("Sections pertinentes en texte brut sans les balises HTML:\n")
for parametre_brut in parametres_bruts:
    texte_affichage = parametre_brut.getText().strip()
    print(texte_affichage)

Sections pertinentes en texte brut sans les balises HTML:

2023-09-21 (Jeu)


Heure EDT
Niveaux (m)
Niveaux (pi)




01:02
0.9
2.9


07:08
3.5
11.4


12:54
1.1
3.5


19:27
4.1
13.4
2023-09-22 (ven)


Heure EDT
Niveaux (m)
Niveaux (pi)




01:55
1.1
3.7


07:59
3.2
10.5


13:40
1.3
4.3


20:24
3.9
12.9
2023-09-23 (sam)


Heure EDT
Niveaux (m)
Niveaux (pi)




03:10
1.3
4.4


09:08
3
9.7


14:47
1.5
4.9


21:36
3.8
12.5
2023-09-24 (dim)


Heure EDT
Niveaux (m)
Niveaux (pi)




04:48
1.4
4.5


10:35
2.9
9.5


16:25
1.6
5.1


23:02
3.9
12.8
2023-09-25 (lun)


Heure EDT
Niveaux (m)
Niveaux (pi)




06:11
1.1
3.7


12:05
3.1
10.2


17:58
1.3
4.3
2023-09-26 (mar)


Heure EDT
Niveaux (m)
Niveaux (pi)




00:22
4.2
13.6


07:11
0.8
2.6


13:11
3.5
11.4


19:06
0.9
3
2023-09-27 (mer)


Heure EDT
Niveaux (m)
Niveaux (pi)




01:26
4.5
14.8


07:58
0.4
1.4


14:01
3.9
12.9


20:01
0.5
1.6


## Affichage en rendu des tableaux HTML de marées

In [124]:
import re
from IPython.display import display, HTML

# Extraction des attributs au moyen d'une expression régulière
# la mise au point de l'expression régulière se fait avec https://regex101.com/

FORME_1 = r"(?:.*\n)<caption>(\d\d\d\d-\d\d-\d\d\s*\(\w*\))\<\/caption>"

for parametre_brut in parametres_bruts:
    titre = re.match(re.compile(FORME_1),str(parametre_brut)).group(1)
    substitution = re.sub(FORME_1, '<table>', str(parametre_brut))
    display(HTML(titre+"\n"+substitution))

Heure EDT,Niveaux (m),Niveaux (pi)
01:02,0.9,2.9
07:08,3.5,11.4
12:54,1.1,3.5
19:27,4.1,13.4


Heure EDT,Niveaux (m),Niveaux (pi)
01:55,1.1,3.7
07:59,3.2,10.5
13:40,1.3,4.3
20:24,3.9,12.9


Heure EDT,Niveaux (m),Niveaux (pi)
03:10,1.3,4.4
09:08,3,9.7
14:47,1.5,4.9
21:36,3.8,12.5


Heure EDT,Niveaux (m),Niveaux (pi)
04:48,1.4,4.5
10:35,2.9,9.5
16:25,1.6,5.1
23:02,3.9,12.8


Heure EDT,Niveaux (m),Niveaux (pi)
06:11,1.1,3.7
12:05,3.1,10.2
17:58,1.3,4.3


Heure EDT,Niveaux (m),Niveaux (pi)
00:22,4.2,13.6
07:11,0.8,2.6
13:11,3.5,11.4
19:06,0.9,3


Heure EDT,Niveaux (m),Niveaux (pi)
01:26,4.5,14.8
07:58,0.4,1.4
14:01,3.9,12.9
20:01,0.5,1.6


## Extraction des données de marées avec des expressions régulières

In [130]:
def extraire_numero_station(texte_entree):
    FORME_1 = r'\s+\((\d*)\)\s+|\s+Pêches\s+et\s+Océans Canada<\/title>'
    return re.search(FORME_1, str(texte_entree)).group(1)

numero_station = extraire_numero_station(dom_page_resultats)
print("Numéro de station:",numero_station)

Numéro de station: 03425


In [125]:
def extraire_fuseau_horaire(texte_entree):
    FORME_2 = r'<th>Heure\s*(\w*)<\/th>'
    return re.search(FORME_2, str(texte_entree)).group(1)

fuseau_horaire = extraire_fuseau_horaire(parametres_bruts[0])
print("Fuseau horaire:",fuseau_horaire)

Fuseau horaire: EDT


In [126]:
from datetime import datetime

def extraire_date(texte_entree):
    FORME_3 = r"(\d\d\d\d)-(\d\d)-(\d\d)"
    resultats_extraction = re.search(re.compile(FORME_3),str(texte_entree))
    # Réécriture de la date en français
    date_str = resultats_extraction.group(3) + "-" + resultats_extraction.group(2) + "-" + resultats_extraction.group(1) 
    return date_str

extraire_date(parametres_bruts[0])

'21-09-2023'

In [127]:
def extraire_donnees_maree(texte_entree):
    FORME_4 = r"<td>(\d*:\d*)<\/td>\n<td>(\d*.\d*)<\/td>"
    resultats_extraction = re.findall(re.compile(FORME_4),str(texte_entree))
    return resultats_extraction

extraire_donnees_maree(parametres_bruts[0])

[('01:02', '0.9'), ('07:08', '3.5'), ('12:54', '1.1'), ('19:27', '4.1')]

## Sauvegarde des données dans le fichier `donnees_marees_formulaire_web.csv`

In [131]:
# Sauvegarder les données dans un fichier .csv
chemin_fichier_sortie = './'
nom_fichier_sortie = "donnees_marees_formulaire_web.csv"
with open(chemin_fichier_sortie+nom_fichier_sortie,'w') as fichier_sortie:
    # Écriture de l'entête du fichier listant les différents attributs
    fichier_sortie.write('localite\tstation\tfuseau_horaire\tdate\theure\thauteur_m\n')
    for parametre_brut in parametres_bruts:
        titre = re.match(re.compile(FORME_1),str(parametre_brut)).group(1)
        date = extraire_date(parametre_brut)
        donnees_marees_liste = extraire_donnees_maree(parametre_brut)
        for donnees_marees in donnees_marees_liste:
            # Écriture des données sur la marée dans le fichier
            fichier_sortie.write(localite+'\t'+numero_station+'\t'+fuseau_horaire+'\t'+date+'\t'
                                 +donnees_marees[0]+'\t'+donnees_marees[1]+'\n')
        
print("Données de marées sauvegardées dans le fichier "+ nom_fichier_sortie)


Données de marées sauvegardées dans le fichier donnees_marees_formulaire_web.csv


## Test de lecture du fichier de données

Vérification que le fichier .csv est correct.

In [133]:
import pandas as pd

nom_fichier_sortie = "donnees_marees_formulaire_web.csv"


donnees_marees_df = pd.read_csv(chemin_fichier_sortie+nom_fichier_sortie,delimiter='\t')
donnees_marees_df

,localite,station,fuseau_horaire,date,heure,hauteur_m
0,Tadoussac,3425,EDT,21-09-2023,01:02,0.9
1,Tadoussac,3425,EDT,21-09-2023,07:08,3.5
2,Tadoussac,3425,EDT,21-09-2023,12:54,1.1
3,Tadoussac,3425,EDT,21-09-2023,19:27,4.1
4,Tadoussac,3425,EDT,22-09-2023,01:55,1.1
5,Tadoussac,3425,EDT,22-09-2023,07:59,3.2
6,Tadoussac,3425,EDT,22-09-2023,13:40,1.3
7,Tadoussac,3425,EDT,22-09-2023,20:24,3.9
8,Tadoussac,3425,EDT,23-09-2023,03:10,1.3
9,Tadoussac,3425,EDT,23-09-2023,09:08,3.0
